# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# Loan CSV file generated from WeatherPy Folder
csv_read = "sanity/city_data.csv"
csv_import = pd.read_csv(csv_read)

# dropping n/a values from DataFrame
city_data = csv_import.dropna()
del city_data["Unnamed: 0"]

# Saving clean data in DataFrame
city_data = pd.DataFrame(city_data)

# Checking DataFrame.
city_data.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,Mataura,85,NZ,1589746747,91,-46.19,168.86,44.83,2.48
1,Tiksi,100,RU,1589746503,95,71.69,128.87,33.10,12.46
2,Severo-Kuril'sk,100,RU,1589746750,93,50.68,156.12,35.28,21.27
3,Novyy Buh,40,UA,1589746751,93,47.69,32.52,51.80,4.47
4,Smithers,90,CA,1589746752,82,54.78,-127.17,55.40,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Configuring gmaps
gmaps.configure(api_key=g_key)

# Setting locations to variable.
locations = city_data[["Lat", "Lng"]]

# Casting Humidity values to floating point numbers.
humidity = city_data["Humidity"].astype(float)

In [15]:
# Heatmap Plot
fig15 = gmaps.figure()

# Creatin the heat layer and saving in heat_map_layer variable.
heat_map_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Passing heat layer, saved in heat_map_layer variable to fig15.
fig15.add_layer(heat_map_layer)

# Figure output
fig15
# plt.savefig("output_plots/Humidity_Heatmap.png")


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
weather_criteria_df = city_data.copy()
weather_criteria_df = weather_criteria_df[weather_criteria_df['Max Temperature'] >= 30]
weather_criteria_df = weather_criteria_df[weather_criteria_df['Max Temperature'] <= 50]  
weather_criteria_df = weather_criteria_df[weather_criteria_df['Wind Speed'] <= 15]
weather_criteria_df = weather_criteria_df[weather_criteria_df['Cloudiness'] == 5]
weather_criteria_df = weather_criteria_df.dropna()
weather_criteria_df = weather_criteria_df.reset_index()
weather_criteria_df

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed
0,525,Poronaysk,5,RU,1589747404,75,49.22,143.12,37.92,13.31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
# list to store hotel results
hotels = []

hotel_df = pd.DataFrame(hotel_df)
# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(weather_criteria_df["City"])):

    lat = hotel_df.loc[city]["Lat"]
    lng = hotel_df.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    google_base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(google_base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Nearest Hotel"] = hotels
hotel_df = pd.DataFrame(hotel_df)
hotel_df.head()

,index,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temperature,Wind Speed,Nearest Hotel
0,525,Poronaysk,5,RU,1589747404,75,49.22,143.12,37.92,13.31,Gostinitsa Sever


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
